In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Fetch Bitcoin data
def get_bitcoin_data():
    end_date = datetime.now()
    start_date = end_date - timedelta(days=730)  # Maximum allowed: 730 days
    
    btc = yf.Ticker("BTC-USD")
    df = btc.history(start=start_date, end=end_date, interval="1d")
    
    # Ensure the index is DatetimeIndex
    df.index = pd.to_datetime(df.index)
    
    return df

# Perform backtest
def backtest(df):
    initial_investment = 1_000_000  # $1 million in Bitcoin
    accumulated_loss = 0

    # Calculate daily returns
    df['Return'] = df['Close'].pct_change()
    
    # Identify deviations of 21 basis points or more
    df['Deviation'] = abs(df['Return']) >= 0.0021
    
    # Calculate absolute value difference and accumulated loss
    for i in range(1, len(df)):
        if df['Deviation'].iloc[i]:
            price_diff = abs(df['Close'].iloc[i] - df['Close'].iloc[i-1])
            bitcoin_amount = initial_investment / df['Close'].iloc[0]
            loss = price_diff * bitcoin_amount
            accumulated_loss += loss
    
    df['Accumulated_Loss'] = accumulated_loss
    
    return df

# Main function
def main():
    # Get Bitcoin data
    btc_data = get_bitcoin_data()
    
    # Perform backtest
    results = backtest(btc_data)
    
    # Print results
    print(results[results['Deviation']])
    
    # Calculate statistics
    total_days = len(results)
    deviation_days = results['Deviation'].sum()
    deviation_percentage = (deviation_days / total_days) * 100
    final_accumulated_loss = results['Accumulated_Loss'].iloc[-1]
    
    print(f"\nTotal days analyzed: {total_days}")
    print(f"Days with 21+ basis point deviation: {deviation_days}")
    print(f"Percentage of days with significant deviation: {deviation_percentage:.2f}%")
    print(f"Total accumulated loss: ${final_accumulated_loss:,.2f}")

if __name__ == "__main__":
    main()

                                   Open          High           Low  \
Date                                                                  
2022-07-10 00:00:00+00:00  21591.080078  21591.080078  20727.123047   
2022-07-11 00:00:00+00:00  20856.353516  20856.353516  19924.539062   
2022-07-12 00:00:00+00:00  19970.474609  20043.445312  19308.531250   
2022-07-13 00:00:00+00:00  19325.972656  20223.052734  18999.953125   
2022-07-14 00:00:00+00:00  20211.466797  20789.894531  19689.257812   
...                                 ...           ...           ...   
2024-07-03 00:00:00+00:00  62034.332031  62187.703125  59419.386719   
2024-07-04 00:00:00+00:00  60147.136719  60399.675781  56777.804688   
2024-07-05 00:00:00+00:00  57022.808594  57497.152344  53717.375000   
2024-07-06 00:00:00+00:00  56659.074219  58472.546875  56038.960938   
2024-07-07 00:00:00+00:00  58239.429688  58371.121094  55793.324219   

                                  Close       Volume  Dividends  Stock Split

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import timedelta

# Download BTC data for the last 10 years
btc = yf.Ticker("BTC-USD")
data = btc.history(period="10y")

# Initialize variables
initial_investment = 1_000_000
purchase_price = data['Close'].iloc[0]
btc_amount = initial_investment / purchase_price
purchase_date = data.index[0]
total_loss = 0

purchase_points = [(purchase_date, purchase_price, btc_amount)]

# Iterate through each day
for date, row in data.iterrows():
    current_price = row['Close']
    
    # Calculate returns from last purchase point
    returns = (current_price - purchase_price) / purchase_price
    
    # Check if deviation is greater than 20 basis points (0.2%)
    if abs(returns) > 0.002:
        # Calculate loss
        
        if current_price < purchase_price:
            loss = current_price + (0.004 * current_price) - purchase_price
        if current_price > purchase_price:
            loss = current_price - (0.004 * current_price) - purchase_price
        
        #loss = abs(current_price - purchase_price) * btc_amount
        total_loss += abs(loss)
        
        # Update purchase point
        purchase_price = current_price
        purchase_date = date
        btc_amount = initial_investment / purchase_price
        
        purchase_points.append((purchase_date, purchase_price, btc_amount))

# Print results
print(f"Total loss: ${total_loss:.2f}")
print(f"Number of purchases: {len(purchase_points)}")
print(f"Final BTC amount: {btc_amount:.8f}")
print(f"Final portfolio value: ${btc_amount * data['Close'].iloc[-1]:.2f}")

# Create a DataFrame of purchase points
purchase_df = pd.DataFrame(purchase_points, columns=['Date', 'Price', 'BTC Amount'])

# Calculate the number of days between purchases
purchase_df['Days Since Last Purchase'] = (purchase_df['Date'] - purchase_df['Date'].shift()).dt.days

# Filter purchase points that skip 5 or more days
skipped_days_df = purchase_df[purchase_df['Days Since Last Purchase'] >= 3].copy()

# Calculate the exact number of days skipped
skipped_days_df['Days Skipped'] = skipped_days_df['Days Since Last Purchase'] - 1

print("\nPurchase Points that Skip 5 or More Days:")
print(skipped_days_df[['Date', 'Price', 'BTC Amount', 'Days Skipped']])

# Calculate statistics on skipped days
avg_skipped_days = skipped_days_df['Days Skipped'].mean()
max_skipped_days = skipped_days_df['Days Skipped'].max()
min_skipped_days = skipped_days_df['Days Skipped'].min()

print(f"\nAverage days skipped: {avg_skipped_days:.2f}")
print(f"Maximum days skipped: {max_skipped_days}")
print(f"Minimum days skipped: {min_skipped_days}")

Total loss: $1260904.78
Number of purchases: 3267
Final BTC amount: 17.35235703
Final portfolio value: $1000000.00

Purchase Points that Skip 5 or More Days:
                          Date         Price   BTC Amount  Days Skipped
240  2015-05-30 00:00:00+00:00    233.345001  4285.499988           4.0
340  2015-09-16 00:00:00+00:00    229.091003  4365.077568           2.0
447  2016-01-12 00:00:00+00:00    435.690002  2295.209884           2.0
499  2016-03-10 00:00:00+00:00    417.131012  2397.328348           2.0
504  2016-03-17 00:00:00+00:00    420.621002  2377.437158           2.0
518  2016-04-05 00:00:00+00:00    424.029999  2358.323710           2.0
552  2016-05-15 00:00:00+00:00    457.567993  2185.467548           2.0
554  2016-05-19 00:00:00+00:00    438.714996  2279.384129           2.0
663  2016-09-18 00:00:00+00:00    609.874023  1639.682888           2.0
733  2016-12-15 00:00:00+00:00    778.088013  1285.201653           2.0
1368 2018-10-24 00:00:00+00:00   6495.839844   153

In [6]:
import yfinance as yf

# Fetch data
btc_data = yf.Ticker("BTC-USD").history(start="2014-01-01")

initial_investment = 1_000_000
short_loss = 0
long_loss = 0
last_long_price = btc_data.iloc[0]['Close']
last_short_price = btc_data.iloc[0]['Close']

for date, row in btc_data.iterrows():
    current_price = row['Close']
    
    # Check short position
    if current_price > last_short_price * 1.002:
        loss = (current_price - last_short_price) * (initial_investment / last_short_price) - (0.004 * current_price)
        short_loss += loss * 0.438
        last_short_price = current_price
    
    # Check long position
    if current_price < last_long_price * 0.998:
        loss = (last_long_price - current_price) * (initial_investment / last_long_price) - (0.004 * current_price)
        long_loss += loss * 0.438
        last_long_price = current_price

print(f"Total short loss: ${short_loss:,.2f}")
print(f"Total long loss: ${long_loss:,.2f}")

Total short loss: $2,296,398.15
Total long loss: $388,387.44


In [7]:
import yfinance as yf

# Fetch data
btc_data = yf.Ticker("BTC-USD").history(start="2014-01-01")

initial_investment = 1_000_000
short_loss = 0
long_loss = 0
last_long_price = btc_data.iloc[0]['Close']
last_short_price = btc_data.iloc[0]['Close']

# New variables to track days between trades
days_since_last_short_trade = 0
days_since_last_long_trade = 0
short_trade_intervals = []
long_trade_intervals = []

for date, row in btc_data.iterrows():
    current_price = row['Close']
    
    # Check short position
    if current_price > last_short_price * 1.002:
        loss = (current_price - last_short_price) * (initial_investment / last_short_price) - (0.004 * current_price)
        short_loss += loss * 0.438
        last_short_price = current_price
        short_trade_intervals.append(days_since_last_short_trade)
        days_since_last_short_trade = 0
    else:
        days_since_last_short_trade += 1
    
    # Check long position
    if current_price < last_long_price * 0.998:
        loss = (last_long_price - current_price) * (initial_investment / last_long_price) - (0.004 * current_price)
        long_loss += loss * 0.438
        last_long_price = current_price
        long_trade_intervals.append(days_since_last_long_trade)
        days_since_last_long_trade = 0
    else:
        days_since_last_long_trade += 1

print(f"Total short loss: ${short_loss:,.2f}")
print(f"Total long loss: ${long_loss:,.2f}")

# Calculate and print statistics about trade intervals
if short_trade_intervals:
    avg_short_interval = sum(short_trade_intervals) / len(short_trade_intervals)
    max_short_interval = max(short_trade_intervals)
    min_short_interval = min(short_trade_intervals)
    print(f"Short trades - Avg interval: {avg_short_interval:.2f} days, Max: {max_short_interval} days, Min: {min_short_interval} days")
else:
    print("No short trades occurred")

if long_trade_intervals:
    avg_long_interval = sum(long_trade_intervals) / len(long_trade_intervals)
    max_long_interval = max(long_trade_intervals)
    min_long_interval = min(long_trade_intervals)
    print(f"Long trades - Avg interval: {avg_long_interval:.2f} days, Max: {max_long_interval} days, Min: {min_long_interval} days")
else:
    print("No long trades occurred")

Total short loss: $2,296,398.15
Total long loss: $388,387.44
Short trades - Avg interval: 23.76 days, Max: 1079 days, Min: 0 days
Long trades - Avg interval: 7.57 days, Max: 72 days, Min: 0 days


In [12]:
import yfinance as yf
from datetime import datetime, timedelta

# Calculate the start date (730 days ago from today)
end_date = datetime.now()
start_date = end_date - timedelta(days=730)

# Fetch hourly data for the last 730 days
btc_data = yf.Ticker("BTC-USD").history(interval="1h", start="2023-01-01", end=end_date)

initial_investment = 1_000_000
short_loss = 0
long_loss = 0
last_long_price = btc_data.iloc[0]['Close']
last_short_price = btc_data.iloc[0]['Close']

# Lists to track trade details
short_trades = []
long_trades = []

for date, row in btc_data.iterrows():
    current_price = row['Close']
    
    # Check short position
    if current_price > last_short_price * 1.002:
        loss = (current_price - last_short_price) * (initial_investment / last_short_price) - (0.004 * current_price)
        short_loss += loss * 0.438
        short_trades.append({
            'date': date.strftime('%Y-%m-%d %H:%M:%S'),
            'entry_price': last_short_price,
            'exit_price': current_price,
            'loss': loss * 0.438
        })
        last_short_price = current_price
    
    # Check long position
    if current_price < last_long_price * 0.998:
        loss = (last_long_price - current_price) * (initial_investment / last_long_price) - (0.004 * current_price)
        long_loss += loss * 0.438
        long_trades.append({
            'date': date.strftime('%Y-%m-%d %H:%M:%S'),
            'entry_price': last_long_price,
            'exit_price': current_price,
            'loss': loss * 0.438
        })
        last_long_price = current_price

print(f"Analysis period: {start_date.date()} to {end_date.date()}")
print(f"Total short loss: ${short_loss:,.2f}")
print(f"Total long loss: ${long_loss:,.2f}")

print("\nFirst 5 Short Trades:")
for trade in short_trades:
    print(f"Date: {trade['date']}, Entry: ${trade['entry_price']:.2f}, Exit: ${trade['exit_price']:.2f}, Loss: ${trade['loss']:.2f}")

print("\nFirst 5 Long Trades:")
for trade in long_trades:
    print(f"Date: {trade['date']}, Entry: ${trade['entry_price']:.2f}, Exit: ${trade['exit_price']:.2f}, Loss: ${trade['loss']:.2f}")

print(f"\nTotal number of short trades: {len(short_trades)}")
print(f"Total number of long trades: {len(long_trades)}")

Analysis period: 2022-07-13 to 2024-07-12
Total short loss: $645,738.96
Total long loss: $0.00

First 5 Short Trades:
Date: 2023-01-01 16:00:00, Entry: $16539.40, Exit: $16572.50, Loss: $847.67
Date: 2023-01-01 19:00:00, Entry: $16572.50, Exit: $16607.92, Loss: $906.87
Date: 2023-01-02 03:00:00, Entry: $16607.92, Exit: $16675.17, Loss: $1744.57
Date: 2023-01-02 07:00:00, Entry: $16675.17, Exit: $16733.32, Loss: $1497.99
Date: 2023-01-04 03:00:00, Entry: $16733.32, Exit: $16837.29, Loss: $2692.02
Date: 2023-01-04 06:00:00, Entry: $16837.29, Exit: $16871.30, Loss: $855.01
Date: 2023-01-04 18:00:00, Entry: $16871.30, Exit: $16957.01, Loss: $2195.45
Date: 2023-01-08 15:00:00, Entry: $16957.01, Exit: $16992.50, Loss: $886.99
Date: 2023-01-08 23:00:00, Entry: $16992.50, Exit: $17085.74, Loss: $2373.53
Date: 2023-01-09 00:00:00, Entry: $17085.74, Exit: $17163.51, Loss: $1963.58
Date: 2023-01-09 03:00:00, Entry: $17163.51, Exit: $17207.60, Loss: $1094.89
Date: 2023-01-09 09:00:00, Entry: $1720